# Feature Engineering

Goal: turn raw hourly prices into useful model features.

We build:
- time features (hour/day/month/weekend)
- seasonality-normalized price (bucket z-score)
- quantile flags (q20/q80, spike)
- autocorrelation (lags)
- short-term dynamics (returns + rolling stats)
- RSI

Note: "Volume fraction + free capacity" comes from the environment during RL, not from the Excel price file.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
TRAIN_PATH = Path("../Data/train.xlsx")  # change if needed
assert TRAIN_PATH.exists(), f"File not found: {TRAIN_PATH.resolve()}"

df = pd.read_excel(TRAIN_PATH)
df["PRICES"] = pd.to_datetime(df["PRICES"])

# the 24 hourly columns (everything except PRICES)
hour_cols = [c for c in df.columns if c != "PRICES"]
print("Rows(days):", len(df), "Hourly columns:", len(hour_cols))
print("First date:", df["PRICES"].min(), "Last date:", df["PRICES"].max())

# melt into long format: one row per hour
long = df.melt(id_vars=["PRICES"], value_vars=hour_cols, var_name="h", value_name="price")

# extract hour number (1..24) from column names
long["hour"] = long["h"].astype(str).str.extract(r"(\d+)").astype(int)

# create timestamp for each hour
long["ts"] = long["PRICES"] + pd.to_timedelta(long["hour"] - 1, unit="h")
long = long.sort_values("ts").set_index("ts")

# numeric price
long["price"] = pd.to_numeric(long["price"], errors="coerce")

long.head()

Rows(days): 1096 Hourly columns: 24
First date: 2007-01-01 00:00:00 Last date: 2009-12-31 00:00:00


,PRICES,h,price,hour
ts,,,,
2007-01-01 00:00:00,2007-01-01,Hour 01,24.31,1
2007-01-01 01:00:00,2007-01-01,Hour 02,24.31,2
2007-01-01 02:00:00,2007-01-01,Hour 03,21.71,3
2007-01-01 03:00:00,2007-01-01,Hour 04,8.42,4
2007-01-01 04:00:00,2007-01-01,Hour 05,0.01,5


## Basic time features (seasonality signals)

Why useful:
- Hour of day matters (night cheap, evening peaks).
- Day of week matters (weekend discount).
- Month matters (seasonality across seasons).

In [3]:
feat = long.copy()

feat["dow"] = feat["PRICES"].dt.dayofweek          # 0=Mon..6=Sun
feat["is_weekend"] = (feat["dow"] >= 5).astype(int)

feat["month"] = feat["PRICES"].dt.month            # 1..12
feat["day_of_year"] = feat["PRICES"].dt.dayofyear  # 1..366
feat["year"] = feat["PRICES"].dt.year

# (optional) simple normalization into [0,1] ranges
feat["hour_n"] = feat["hour"] / 24.0
feat["dow_n"] = feat["dow"] / 6.0
feat["month_n"] = feat["month"] / 12.0
feat["doy_n"] = feat["day_of_year"] / 366.0

feat[["price","hour","dow","is_weekend","month","day_of_year","year"]].head()

,price,hour,dow,is_weekend,month,day_of_year,year
ts,,,,,,,
2007-01-01 00:00:00,24.31,1,0,0,1,1,2007
2007-01-01 01:00:00,24.31,2,0,0,1,1,2007
2007-01-01 02:00:00,21.71,3,0,0,1,1,2007
2007-01-01 03:00:00,8.42,4,0,0,1,1,2007
2007-01-01 04:00:00,0.01,5,0,0,1,1,2007


## Cyclical encodings (sin/cos)

Why useful:
- hour 23 and hour 0 are close, not far.
- sin/cos makes time "circular".

If your team does NOT want cycles, you can skip this cell.

In [4]:
def add_cyclical(df, col, period, prefix):
    x = 2 * np.pi * df[col] / period
    df[f"{prefix}_sin"] = np.sin(x)
    df[f"{prefix}_cos"] = np.cos(x)
    return df

feat = add_cyclical(feat, "hour", 24, "hour")
feat = add_cyclical(feat, "dow", 7, "dow")
feat = add_cyclical(feat, "month", 12, "month")
feat = add_cyclical(feat, "day_of_year", 365, "doy")

feat[[c for c in feat.columns if "sin" in c or "cos" in c]].head()

,hour_sin,hour_cos,dow_sin,dow_cos,month_sin,month_cos,doy_sin,doy_cos
ts,,,,,,,,
2007-01-01 00:00:00,0.258819,0.965926,0.0,1.0,0.5,0.866025,0.017213,0.999852
2007-01-01 01:00:00,0.500000,0.866025,0.0,1.0,0.5,0.866025,0.017213,0.999852
2007-01-01 02:00:00,0.707107,0.707107,0.0,1.0,0.5,0.866025,0.017213,0.999852
2007-01-01 03:00:00,0.866025,0.500000,0.0,1.0,0.5,0.866025,0.017213,0.999852
2007-01-01 04:00:00,0.965926,0.258819,0.0,1.0,0.5,0.866025,0.017213,0.999852


## Price normalized by (hour, weekend) bucket

Why useful:
- Prices have strong intraday pattern and weekend discount.
- We normalize price inside each bucket (hour + weekend).
- This helps the model learn "is this price high for this hour?" instead of absolute level.

In [5]:
bucket = feat.groupby(["hour", "is_weekend"])["price"]
bucket_mean = bucket.transform("mean")
bucket_std = bucket.transform("std").replace(0, np.nan)

feat["price_bucket_z"] = (feat["price"] - bucket_mean) / (bucket_std + 1e-9)
feat["price_bucket_z"] = feat["price_bucket_z"].fillna(0.0)

feat[["price","hour","is_weekend","price_bucket_z"]].head()

,price,hour,is_weekend,price_bucket_z
ts,,,,
2007-01-01 00:00:00,24.31,1,0,-0.806031
2007-01-01 01:00:00,24.31,2,0,-0.573271
2007-01-01 02:00:00,21.71,3,0,-0.518516
2007-01-01 03:00:00,8.42,4,0,-1.180272
2007-01-01 04:00:00,0.01,5,0,-1.700637


## Quantile features (train-only) and spike flags

Why useful:
- Similar idea as quantile baseline, but as features.
- below_q20 means "cheap for this hour/weekend".
- above_q80 means "expensive for this hour/weekend".
- spike flags help catch rare extreme events.

In [6]:
q20 = feat.groupby(["hour","is_weekend"])["price"].transform(lambda s: s.quantile(0.20))
q80 = feat.groupby(["hour","is_weekend"])["price"].transform(lambda s: s.quantile(0.80))

feat["below_q20"] = (feat["price"] < q20).astype(int)
feat["above_q80"] = (feat["price"] > q80).astype(int)

# global spikes (very rare extremes)
q99 = feat["price"].quantile(0.99)
q01 = feat["price"].quantile(0.01)
feat["spike_up_99"] = (feat["price"] >= q99).astype(int)
feat["spike_down_01"] = (feat["price"] <= q01).astype(int)

feat[["price","below_q20","above_q80","spike_up_99","spike_down_01"]].head()

,price,below_q20,above_q80,spike_up_99,spike_down_01
ts,,,,,
2007-01-01 00:00:00,24.31,0,0,0,0
2007-01-01 01:00:00,24.31,0,0,0,0
2007-01-01 02:00:00,21.71,0,0,0,0
2007-01-01 03:00:00,8.42,1,0,0,0
2007-01-01 04:00:00,0.01,1,0,0,1


## Autocorrelation features (lags) + short-term dynamics

Why useful:
- Prices are persistent (lag-1 high).
- Yesterday same hour matters (lag-24).
- Rolling stats capture short-term trend and volatility.

In [7]:
# price lags (1/2/3/6/12/24)
for L in [1,2,3,6,12,24]:
    feat[f"price_lag_{L}"] = feat["price"].shift(L)

# 1-step return / delta
feat["delta_1h"] = feat["price"].diff(1)
feat["return_1h"] = feat["price"].pct_change(1).replace([np.inf, -np.inf], np.nan).fillna(0.0)

# rolling stats over 6h and 24h (on price and delta)
for W in [6, 24]:
    feat[f"roll_mean_{W}h"] = feat["price"].rolling(W).mean()
    feat[f"roll_std_{W}h"]  = feat["price"].rolling(W).std()
    feat[f"roll_min_{W}h"]  = feat["price"].rolling(W).min()
    feat[f"roll_max_{W}h"]  = feat["price"].rolling(W).max()

    feat[f"delta_roll_mean_{W}h"] = feat["delta_1h"].rolling(W).mean()
    feat[f"delta_roll_std_{W}h"]  = feat["delta_1h"].rolling(W).std()

feat[["price","price_lag_1","price_lag_24","delta_1h","roll_mean_24h","roll_std_24h"]].head(30)

,price,price_lag_1,price_lag_24,delta_1h,roll_mean_24h,roll_std_24h
ts,,,,,,
2007-01-01 00:00:00,24.31,NaN,NaN,NaN,NaN,NaN
2007-01-01 01:00:00,24.31,24.31,NaN,0.00,NaN,NaN
2007-01-01 02:00:00,21.71,24.31,NaN,-2.60,NaN,NaN
2007-01-01 03:00:00,8.42,21.71,NaN,-13.29,NaN,NaN
2007-01-01 04:00:00,0.01,8.42,NaN,-8.41,NaN,NaN
2007-01-01 05:00:00,0.01,0.01,NaN,0.00,NaN,NaN
2007-01-01 06:00:00,0.02,0.01,NaN,0.01,NaN,NaN
2007-01-01 07:00:00,0.01,0.02,NaN,-0.01,NaN,NaN
2007-01-01 08:00:00,0.01,0.01,NaN,0.00,NaN,NaN


## RSI feature

Why useful:
- RSI is a "momentum" score from 0 to 100.
- Low RSI = price has been falling (possibly cheap).
- High RSI = price has been rising (possibly expensive).

We compute RSI over 14 hours (common default).

In [8]:
def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)

    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()

    rs = avg_gain / (avg_loss + 1e-12)
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50.0)

feat["rsi_14"] = rsi(feat["price"], 14)
feat["rsi_14_n"] = feat["rsi_14"] / 100.0  # normalize to 0..1

feat[["price","rsi_14","rsi_14_n"]].head(30)

,price,rsi_14,rsi_14_n
ts,,,
2007-01-01 00:00:00,24.31,50.000000,0.500000
2007-01-01 01:00:00,24.31,50.000000,0.500000
2007-01-01 02:00:00,21.71,50.000000,0.500000
2007-01-01 03:00:00,8.42,50.000000,0.500000
2007-01-01 04:00:00,0.01,50.000000,0.500000
2007-01-01 05:00:00,0.01,50.000000,0.500000
2007-01-01 06:00:00,0.02,50.000000,0.500000
2007-01-01 07:00:00,0.01,50.000000,0.500000
2007-01-01 08:00:00,0.01,50.000000,0.500000


## Final dataset (drop early NaNs caused by lags/rolling) + export

We drop rows where lag/rolling features are NaN (first ~24 hours).
Then we save a clean feature table.

In [9]:
# pick columns you want (you can add/remove)
feature_cols = [
    # raw + time
    "price","hour","dow","is_weekend","month","day_of_year","year",
    "hour_n","dow_n","month_n","doy_n",

    # cyclical (optional; remove if you skipped)
    "hour_sin","hour_cos","dow_sin","dow_cos","month_sin","month_cos","doy_sin","doy_cos",

    # seasonality normalization
    "price_bucket_z",

    # quantile flags
    "below_q20","above_q80","spike_up_99","spike_down_01",

    # lags + dynamics
    "price_lag_1","price_lag_2","price_lag_3","price_lag_6","price_lag_12","price_lag_24",
    "delta_1h","return_1h",
    "roll_mean_6h","roll_std_6h","roll_min_6h","roll_max_6h",
    "roll_mean_24h","roll_std_24h","roll_min_24h","roll_max_24h",
    "delta_roll_mean_6h","delta_roll_std_6h",
    "delta_roll_mean_24h","delta_roll_std_24h",

    # RSI
    "rsi_14","rsi_14_n",
]

# keep only existing columns (in case you removed cyclical)
feature_cols = [c for c in feature_cols if c in feat.columns]

X = feat[feature_cols].copy()

# drop rows with NaNs created by lags/rolling
X_clean = X.dropna().copy()
print("Rows before:", len(X), "Rows after dropna:", len(X_clean))

# save
out_path = Path("train_features.parquet")
X_clean.to_parquet(out_path)
print("Saved:", out_path.resolve())

X_clean.head()

Rows before: 26304 Rows after dropna: 26280
Saved: /Users/chibi/Desktop/Github/reinforcement_learning_project/Code/train_features.parquet


,price,hour,dow,is_weekend,month,day_of_year,year,hour_n,dow_n,month_n,...,roll_mean_24h,roll_std_24h,roll_min_24h,roll_max_24h,delta_roll_mean_6h,delta_roll_std_6h,delta_roll_mean_24h,delta_roll_std_24h,rsi_14,rsi_14_n
ts,,,,,,,,,,,,,,,,,,,,,
2007-01-02 00:00:00,16.01,1,1,0,1,2,2007,0.041667,0.166667,0.083333,...,16.658750,13.333071,0.01,37.99,-3.663333,7.105459,-0.345833,7.686734,54.187092,0.541871
2007-01-02 01:00:00,11.00,2,1,0,1,2,2007,0.083333,0.166667,0.083333,...,16.104167,13.277680,0.01,37.99,-3.685000,7.110108,-0.554583,7.744744,50.833087,0.508331
2007-01-02 02:00:00,9.01,3,1,0,1,2,2007,0.125000,0.166667,0.083333,...,15.575000,13.297610,0.01,37.99,-4.830000,5.906197,-0.529167,7.738738,43.029770,0.430298
2007-01-02 03:00:00,7.50,4,1,0,1,2,2007,0.166667,0.166667,0.083333,...,15.536667,13.320439,0.01,37.99,-4.250000,6.056306,-0.038333,7.252486,46.163819,0.461638
2007-01-02 04:00:00,9.00,5,1,0,1,2,2007,0.208333,0.166667,0.083333,...,15.911250,12.987063,0.01,37.99,-4.580000,5.586598,0.374583,7.033944,46.485473,0.464855
